### Documents
Langchain implements a Document abstraction, which is intended to repersent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content
- metadata: a dict containing arbitary metadata. The metadata attribute can capture information about the source of the document, its relationship to the other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [14]:
from langchain_core.documents import Document

documents=[
    Document(
        page_content="Dogs are great companions, known for their loyalty and frienliness",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source":"fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source":"bird-pets-doc"},
    ),
]

In [15]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [26]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model="openai/gpt-oss-20b")

In [17]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings=OllamaEmbeddings(
    model="gemma:2b"
)

In [18]:
## Vector Stores
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents,embedding=embeddings)
vectorstore

In [19]:
vectorstore.similarity_search("cat")

[Document(id='f0794191-67c1-4835-9348-a9712ea01727', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness,'),
 Document(id='79060a2d-1d21-4bcd-be06-61a20c23f677', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness'),
 Document(id='47452da0-76b2-4857-b20a-28aa24ecec4f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='cb5f4cd2-2fb1-4dbf-bb3e-72b7025090fc', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [20]:
## Async query
await vectorstore.asimilarity_search("cat")

[Document(id='f0794191-67c1-4835-9348-a9712ea01727', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness,'),
 Document(id='79060a2d-1d21-4bcd-be06-61a20c23f677', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness'),
 Document(id='47452da0-76b2-4857-b20a-28aa24ecec4f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='cb5f4cd2-2fb1-4dbf-bb3e-72b7025090fc', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [21]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='f0794191-67c1-4835-9348-a9712ea01727', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness,'),
  4974.32958984375),
 (Document(id='79060a2d-1d21-4bcd-be06-61a20c23f677', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness'),
  5106.1484375),
 (Document(id='47452da0-76b2-4857-b20a-28aa24ecec4f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  5189.09033203125),
 (Document(id='cb5f4cd2-2fb1-4dbf-bb3e-72b7025090fc', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  5189.09033203125)]

### Retrievers

Langchain vectorstore objcets do not subclass Runnable, and so cannot immediately be integrated into Langchain Expression Language chains.

Langchain Retrievers are runnables, so they implement a standard set of methods and area designed to be incorpoated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retreiever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. below we will build one around the similarity_search method:

In [22]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retreiever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retreiever.batch(["cat","dog"])

[[Document(id='f0794191-67c1-4835-9348-a9712ea01727', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness,')],
 [Document(id='f0794191-67c1-4835-9348-a9712ea01727', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness,')]]

In [24]:
retreiever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retreiever.batch(["parrot","dog"])

[[Document(id='47452da0-76b2-4857-b20a-28aa24ecec4f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')],
 [Document(id='f0794191-67c1-4835-9348-a9712ea01727', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and frienliness,')]]

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt=ChatPromptTemplate.from_messages([("human",message)])

rag_chain={"context":retreiever,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("Tell me about dogs")
print(response.content)

Dogs are great companions, known for their loyalty and friendliness.
